# Create the network in Topogram

In [34]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
import csv
import networkx as nx
from collections import Counter

from topogram_client import TopogramAPIClient
import logging 

In [42]:
#network
def create_graph( fichier_nodes, fichier_edges ):
    tmp_G = nx.Graph()

    with open( os.path.join(os.getcwd(), fichier_nodes), "r") as f :
        reader = csv.DictReader(f) 
        for line in reader:
            tmp_G.add_node(line["id"], { 
                "name": line["name"],
                "start" : line["start"], 
                "end" : line["end"], 
                "axe" :line["axe"], 
                "type" : line["type"]
            })

    print "%s nodes"%len(tmp_G.nodes())

    with open( os.path.join(os.getcwd(), fichier_edges), "r") as f :
        reader = csv.DictReader(f) 
        for line in reader:
            if line["source"] in tmp_G.nodes() and line["target"] in tmp_G.nodes() :
                tmp_G.add_edge(line["source"], line["target"], { "name" : line["name"], "type" : line["type"] })

    print "%s edges"%len(tmp_G.edges())
    print "%s types of nodes "%Counter( [ n[1]["type"] for n in tmp_G.nodes(data=True) ])
    return tmp_G

In [58]:
fichier_nodes = "../results/network/ARC5_no_project_nodes_ok.csv"
fichier_edges = "../results/network/ARC5_no_project_edges.csv"

# create Graph
G = create_graph( fichier_nodes, fichier_edges )

print G.edges(data=True)[0]
# for e in G.nodes(data=True) : print e
starts = [ e for e in G.nodes(data=True) if e[1]["start"] != ""]
print starts
print Counter(starts)

# print "%s / %s"%( len(hasEndStart),len(edges) )

258 nodes
1140 edges
Counter({'creation': 59, 'laboratoire': 55, 'etablissement': 40, 'ville': 37, 'patrimoine': 30, 'm\xc3\xa9diation': 15, 'enseignement': 11, 'ecole-doctorale': 8, 'cst': 3}) types of nodes 
('observatoire-des-politiques-culturelles', 'pacte-umr-5194', {'type': 'projects', 'name': 'projets ou th\xc3\xa8ses en commun'})
[]
Counter()


## Import data into Topogram

Using the API client

In [41]:
TOPOGRAM_URL = "https://app.topogram.io"
USER = "arc5@arc5.com"
PASSWORD = "culture&recherche"

# set verbose log
logging.getLogger('topogram-client').setLevel(logging.DEBUG)

# connect to the topogram instance 
topogram = TopogramAPIClient(TOPOGRAM_URL)

# topogram.create_user(USER, PASSWORD)
topogram.user_login(USER, PASSWORD)

r = topogram.create_topogram("ARC 5 - Collaborations Culture / Recherche en Rhône-Alpes")
print r
topogram_ID = r["data"][0]["_id"]


# delete existing nodes
# existing_nodes = topogram.get_nodes(topogram_ID)["data"]
# existing_edges = topogram.get_edges(topogram_ID)["data"]
# print "%s existing edges, %s existing nodes"%(len(existing_edges), len(existing_nodes))
# topogram.delete_nodes([n["_id"] for n in existing_nodes])
# topogram.delete_edges([n["_id"] for n in existing_edges])


# create the graph
nodes = []
for n in G.nodes(data=True): 
    node = n[1]
    node["id"] = n[0]
    node["group"] = n[1]["type"]
    nodes.append(node)

print nodes[0]

edges = []
for e in G.edges(data=True): 
    edge = e[2]
    edge["source"] = e[0]
    edge["target"] = e[1]
    edges.append(edge)

print edges[0]
    
r = topogram.create_nodes(topogram_ID, nodes)
r = topogram.create_edges(topogram_ID, edges)

{u'status': u'error', 'status_code': 200, u'message': u'A topogram with the same name already exists', u'data': [{u'sharedPublic': False, u'name': u'ARC 5 - Collaborations Culture / Recherche en Rh\xf4ne-Alpes', u'owner': u'FWGtG3EXa2F7nZRKp', u'_id': u'3Fep7oZAFjqBnHLQR', u'slug': u'arc-5-collaborations-culture-recherche-en-rhne-alpes', u'createdAt': u'2016-09-12T04:25:24.618Z'}]}
{'end': '', 'name': 'Observatoire des politiques culturelles', 'start': '', 'axe': '', 'group': 'm\xc3\xa9diation', 'type': 'm\xc3\xa9diation', 'id': 'observatoire-des-politiques-culturelles'}
{'source': 'observatoire-des-politiques-culturelles', 'type': 'projects', 'name': 'projets ou th\xc3\xa8ses en commun', 'target': 'pacte-umr-5194'}
